In [ ]:
# Calcula la temperatura media anual.

import os

import pandas as pd
import numpy as np

import xarray as xr

from scipy import integrate

In [ ]:
path_d = "../../../resultados/CHIRTS/"
path_r = "../../../resultados/vars/"

# Variables de CHIRTS.
vars = ["Tmax", "Tmin", "Tmean"]

# Si no existe la carpeta, la crea.
if not os.path.exists(path_r):
    os.mkdir(path_r)

CDD = True
varname = "CDD_mean"

In [ ]:
def zero(a, b, c):
    return ( 12 / np.pi *
        np.arccos( ( a + b - 2 * c )
        / ( a - b ) )
        )

def day_curve(x, a, b, c):
    return ( ( - 0.5 * ( a - b )
        * np.cos( x * np.pi / 12 ) )
        + ( a - b ) / 2 + b - c
        )

def degree_day(ini, fin, a, b, c):
    return integrate.quad(day_curve,
        ini, fin,
        args = (a, b, c) )

vec_degree = np.vectorize(degree_day)
vec_zero = np.vectorize(zero)

In [ ]:
a = np.array(([12, 13], [14, 15]))
a = np.repeat(np.expand_dims(a, 2), 2, 2)
b = np.array(([1, 2], [3, 4]))
b = np.repeat(np.expand_dims(b, 2), 2, 2)
c = np.array(([5, 5], [5, 5]))
c = np.repeat(np.expand_dims(c, 2), 2, 2)

#vec_zero(a, b, c)

x = vec_degree( 0,
     vec_zero(a, b, c),
     a, b, c )[0]

x

In [ ]:
j = 1995

ds_mean = ( ( xr.open_dataset(path_d + "mexico_Tmean.nc")
    .isel(time = j - 1995) )
    .rename(Tmean = "degree_day") 
    .to_array() )
ds_dummy = xr.zeros_like(ds_mean)
ds_plus = xr.zeros_like(ds_mean)
ds_minus = xr.zeros_like(ds_mean)

with ( xr.open_dataset( path_d + vars[0] + "/mexico_" 
    + vars[0] + "." + str(j) + ".nc" )
    .rename(Tmax = "degree_day")
    .to_array() ) as ds_max:
    # Establecemos np.nan como valor sin datos.
    ds_max = ds_max.where(ds_max > -9000)
    with ( xr.open_dataset( path_d + vars[1] + "/mexico_" 
        + vars[1] + "." + str(j) + ".nc" )
        .rename(Tmin = "degree_day")
        .to_array() )  as ds_min:
        # Establecemos np.nan como valor sin datos.
        ds_min = ds_min.where(ds_min > -9000)
        ds_dummy = ds_dummy.where(ds_max > ds_mean, -1)
        ds_dummy = ds_dummy.where(ds_min < ds_mean, -2)
        ds_dummy = ds_dummy.where(ds_dummy < 0, 
            vec_zero(ds_max, ds_min, ds_mean) )
        ds_plus = ds_dummy.where(
            ds_dummy != -2,
            vec_degree( 0, 12, ds_max,
                ds_min, ds_mean )
            )
        ds_minus = ds_dummy.where(
            ds_dummy != -1, 
            vec_degree( 0, 12, ds_max,
                ds_min, ds_mean )
            )
        ds_minus = ds_dummy.where(
            ds_dummy < 0, 
            vec_degree( 
                ds_dummy(ds_dummy > -1), 12,
                ds_max, ds_min, ds_mean )
            )
        ds_minus = ds_dummy.where(
            ds_dummy < 0,
            vec_degree( day_curve,
                0, ds_dummy(ds_dummy > -1),
                ds_max, ds_min, ds_mean )
            )

if CDD: ds = ds_plus
else: ds = ds_minus

# Si no existe la carpeta, la crea.
if not os.path.exists(path_r + varname):
    os.mkdir(path_r + varname)
ds.to_netcdf(path_r + varname + + "/mexico_" 
    + varname + "." + str(j) + ".nc" )

In [ ]:
j = 1995

ds_mean = ( ( xr.open_dataset(path_d + "mexico_Tmean.nc")
    .isel(time = j - 1995) )
    .rename(Tmean = "degree_day") 
    .to_array() )
ds_dummy = xr.zeros_like(ds_mean)
ds_plus = xr.zeros_like(ds_mean)
ds_minus = xr.zeros_like(ds_mean)

with ( xr.open_dataset( path_d + vars[0] + "/mexico_" 
    + vars[0] + "." + str(j) + ".nc" )
    .rename(Tmax = "degree_day")
    .to_array() ) as ds_max:
    # Establecemos np.nan como valor sin datos.
    ds_max = ds_max.where(ds_max > -9000)
    with ( xr.open_dataset( path_d + vars[1] + "/mexico_" 
        + vars[1] + "." + str(j) + ".nc" )
        .rename(Tmin = "degree_day")
        .to_array() )  as ds_min:
        # Establecemos np.nan como valor sin datos.
        ds_min = ds_min.where(ds_min > -9000)
        ds_vec = xr.DataArray(
            vec_zero(ds_max, ds_min, ds_mean),
            dims = ds_max.dims, coords = ds_max.coords 
            )
        ds_dummy = ds_vec.where(ds_max > ds_mean, -1)
        ds_dummy = ds_dummy.where(ds_min < ds_mean, -2)

In [ ]:
ds_plus = vec_degree( 0, 12, ds_max,
                ds_min, ds_mean )[0]

'''
ds_plus = ds_dummy.where(
            ds_dummy != -2,
            vec_degree( 0, 12, ds_max,
                ds_min, ds_mean )
            )

ds_minus = ds_dummy.where(
            ds_dummy != -1, 
            vec_degree( 0, 12, ds_max,
                ds_min, ds_mean )
            )
ds_minus = ds_dummy.where(
            ds_dummy < 0, 
            vec_degree( 
                ds_dummy.where(ds_dummy > -1), 12,
                ds_max, ds_min, ds_mean )
            )
ds_minus = ds_dummy.where(
            ds_dummy < 0,
            vec_degree( day_curve,
                0, ds_dummy.where(ds_dummy > -1),
                ds_max, ds_min, ds_mean )
            )
'''

In [ ]:
ds_dummy